# Analisa informações gerais dos arquivos de Twitter #

In [ ]:
#Arquivo que analisa o volume de tweets de cada série

In [ ]:
#Bibliotecas
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path
from IPython.core.interactiveshell import InteractiveShell
from datetime import date

#InteractiveShell.ast_node_interactivity = 'all'

In [ ]:
nomes_series = pd.read_csv('input_twitter_analysis.csv')
nomes_series['data_estreia'] = pd.to_datetime(nomes_series['data_estreia'], format="%Y-%m-%d", errors = 'coerce')
print(nomes_series['data_estreia'].dtypes)
nomes_series

In [ ]:
def droparDup(dataframe):
    dataframe.drop_duplicates(subset=['id'], inplace=True)
    
def formatarData(dataframe):
    dataframe['date'] = pd.to_datetime(dataframe['date'], format="%Y-%m-%d", errors = 'coerce')

In [ ]:
for serie in nomes_series['nome']:
    df_serie = pd.read_csv('series_dataset/'+serie+'.csv', error_bad_lines=False)
    df_serie_pre = pd.read_csv('series_dataset/'+serie+'-pre.csv', error_bad_lines=False)
    df_serie = pd.concat([df_serie, df_serie_pre])
    droparDup(df_serie)
    
    print(f"{serie}")
    df_serie.to_csv(f"series_dataset_completo/{serie}_completo.csv", index=False)

In [ ]:
def filtrarDatas(dataframe, data_inicial, data_final):
    filtro_dados = (dataframe["date"] >  data_inicial) & (dataframe["date"] < data_final)
    #print(dataframe[filtro_dados])
    return dataframe[filtro_dados].shape[0]

In [ ]:
def countPre(dataframe, data_estreia):
    data_final = data_estreia - pd.DateOffset(days=3)
    data_inicial = data_estreia - pd.DateOffset(days=76)
    #print(f"Count Pre: Final: {data_final}, Inicial: {data_inicial}")
    
    return filtrarDatas(dataframe, data_inicial, data_final)

In [ ]:
def countDurante(dataframe, data_estreia):
    data_final = data_estreia + pd.DateOffset(days=4)
    data_inicial = data_estreia - pd.DateOffset(days=4)
    #print(f"Count Durante: Final: {data_final}, Inicial: {data_inicial}")
    
    return filtrarDatas(dataframe, data_inicial, data_final)

In [ ]:
def countPos(dataframe, data_estreia):
    data_final = data_estreia + pd.DateOffset(days=151)
    data_inicial = data_estreia + pd.DateOffset(days=3)
    #print(f"Count Pos: Final: {data_final}, Inicial: {data_inicial}")
    
    return filtrarDatas(dataframe, data_inicial, data_final)

In [ ]:
def countTotal(dataframe, data_estreia):
    data_final = data_estreia + pd.DateOffset(days=151)
    data_inicial = data_estreia - pd.DateOffset(days=76)
    #print(f"Count Total: Final: {data_final}, Inicial: {data_inicial}")
    
    return filtrarDatas(dataframe, data_inicial, data_final)

In [ ]:
lista_df = []
for index, linha in nomes_series.iterrows():
    row = []
    df_serie = pd.read_csv(f"series_dataset_completo/{linha['nome']}_completo.csv")
    #df_serie[df_serie["id"].duplicated()]
    formatarData(df_serie)
    
    row.append(linha['nome'])
    row.append(linha['data_estreia']) 
    row.append(countPre(df_serie, linha['data_estreia']))
    row.append(countDurante(df_serie,linha['data_estreia']))
    row.append(countPos(df_serie, linha['data_estreia']))
    row.append(countTotal(df_serie, linha['data_estreia']))
    #print("\n\n\n")
    
    lista_df.append(row)

df = pd.DataFrame(lista_df, columns = ['Nome', 'DataEstreia', 'CountPre', 'CountDurante', 'CountPos', 'CountTotal'])
df

In [ ]:
df.to_csv(f"series_dataset_count/dataset_count.csv", index=False)

In [ ]:
lista_df[1].head()

In [ ]:
for df in lista_df:
    setDataframe(df)

In [ ]:
# todos_df.groupby(["nome_serie", "date"])["date"].count()

## Quantificar hashtags associadas ##

In [ ]:
df_hash = lista_df[1][lista_df[1]["hashtags"].notna()]
# lista_df[1][lista_df[1]["hashtags"].isna()]
# lista_df[1]["hashtags"].isna().sum()

In [ ]:
df_hash["hashtags"] = df_hash["hashtags"].apply(lambda x: eval(str(x)))
s = df_hash["hashtags"].explode()
counts = s.value_counts()
counts.to_csv('hashtag.csv',index=True)

## Visualizar série temporal ##

In [ ]:
#filtro_dados = (df_TheWitcher["date"] > pd.Timestamp(2020, 1, 1)) & (df_TheWitcher["date"] <  pd.Timestamp(2020, 2, 8)

#Criar tuplas de visualização
tuplas_vis = []
for df in lista_df:

    data_final = df['date'].iloc[0] - pd.DateOffset(days=210)
    data_inicial = df['date'].iloc[-1]

    filtro_dados = (df["date"] >  data_inicial) & (df["date"] < data_final)
    y = df[filtro_dados]['date'].value_counts()
    x = df[filtro_dados]['date'].value_counts().index.tolist()

    tuplas_vis.append((x,y))

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10), sharey=False)

sns.lineplot(ax=axes[0,0], x = tuplas_vis[0][0], y = tuplas_vis[0][1], color="salmon")
axes[0,0].tick_params(rotation=90, axis='x')
axes[0,0].set_title(nomes_series[0])
axes[0,0].set_ylabel('Número Total de Twitts')
axes[0,0].set_xlabel('Dia')

sns.lineplot(ax=axes[0,1], x = tuplas_vis[1][0], y = tuplas_vis[1][1], color="salmon")
axes[0,1].tick_params(rotation=90, axis='x')
axes[0,1].set_title(nomes_series[1])
axes[0,1].set_ylabel('Número Total de Twitts')
axes[0,1].set_xlabel('Dia')

sns.lineplot(ax=axes[1,0], x = tuplas_vis[2][0], y = tuplas_vis[2][1], color="salmon")
axes[1,0].tick_params(rotation=90, axis='x')
axes[1,0].set_title(nomes_series[2])
axes[1,0].set_ylabel('Número Total de Twitts')
axes[1,0].set_xlabel('Dia')

sns.lineplot(ax=axes[1,1], x = tuplas_vis[3][0], y = tuplas_vis[3][1], color="salmon")
axes[1,1].tick_params(rotation=90, axis='x')
axes[1,1].set_title(nomes_series[3])
axes[1,1].set_ylabel('Número Total de Twitts')
axes[1,1].set_xlabel('Dia')
